<h1>
<hr style=" border:none; height:3px;">
<center>Main Workflow - Seamless Stable Diffusion CS</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE | Clément VERON | Edouard SEGUIER</center></h4>

# I. Imports

In [16]:
import cv2
import numpy as np

# II. Observation and Blending

<div class="alert alert-block alert-info"> After conducting tests, we realized that some additions were somewhat weird and unrealistic compared to the original image. For instance, adding headphones to a regular photo consistently (across three attempts) resulted in an outlier (very strange style) or half-headphones with poorly matched colors. The idea, therefore, is to add a <b>blending / filtering</b> step to the pipeline so that the headphones integrate better into the photo (aiming for realism and seamless integration).</div>

In [17]:
original_img = cv2.imread('sylvie.jpg')
modified_img = cv2.imread('headphones.png')

if original_img.shape[:2] != modified_img.shape[:2]:
    modified_img = cv2.resize(modified_img, (original_img.shape[1], original_img.shape[0]))

diff_mask = cv2.absdiff(original_img, modified_img)
gray_diff_mask = cv2.cvtColor(diff_mask, cv2.COLOR_BGR2GRAY)
_, binary_diff_mask = cv2.threshold(gray_diff_mask, 30, 255, cv2.THRESH_BINARY)

x, y, w, h = cv2.boundingRect(binary_diff_mask)
center = (x + w // 2, y + h // 2)

original_img_lab = cv2.cvtColor(original_img, cv2.COLOR_BGR2LAB)
modified_img_lab = cv2.cvtColor(modified_img, cv2.COLOR_BGR2LAB)

l_orig, a_orig, b_orig = cv2.split(original_img_lab)
l_mod, a_mod, b_mod = cv2.split(modified_img_lab)

a_mod = cv2.addWeighted(a_mod, 0.9, a_orig, 0.1, 0)
b_mod = cv2.addWeighted(b_mod, 0.9, b_orig, 0.1, 0)

modified_img_lab = cv2.merge((l_mod, a_mod, b_mod))
modified_img = cv2.cvtColor(modified_img_lab, cv2.COLOR_LAB2BGR)

mask = cv2.absdiff(original_img, modified_img)
gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
_, binary_mask = cv2.threshold(gray_mask, 30, 255, cv2.THRESH_BINARY)

# Crop the mask to focus only on the bounding box region
binary_mask_cropped = np.zeros_like(binary_mask)
binary_mask_cropped[y:y+h, x:x+w] = binary_mask[y:y+h, x:x+w]

# Smooth transitions but only on modified regions (so that other parts are not modified)
feathered_mask = cv2.GaussianBlur(binary_mask_cropped, (15, 15), 0)

blended_img = cv2.seamlessClone(modified_img, original_img, feathered_mask, center, cv2.NORMAL_CLONE)

cv2.imwrite('blended_output_v1.jpg', blended_img)
cv2.imshow('Blended Image', blended_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
original_img = cv2.imread('sylvie.jpg')
modified_img = cv2.imread('headphones.png')

if original_img.shape[:2] != modified_img.shape[:2]:
    modified_img = cv2.resize(modified_img, (original_img.shape[1], original_img.shape[0]))

diff_mask = cv2.absdiff(original_img, modified_img)
gray_diff_mask = cv2.cvtColor(diff_mask, cv2.COLOR_BGR2GRAY)
_, binary_diff_mask = cv2.threshold(gray_diff_mask, 30, 255, cv2.THRESH_BINARY)

# Morphological operations to refine the mask
kernel = np.ones((5, 5), np.uint8)
binary_diff_mask = cv2.erode(binary_diff_mask, kernel, iterations=1)
binary_diff_mask = cv2.dilate(binary_diff_mask, kernel, iterations=1)

x, y, w, h = cv2.boundingRect(binary_diff_mask)
center = (x + w // 2, y + h // 2)

original_img_lab = cv2.cvtColor(original_img, cv2.COLOR_BGR2LAB)
modified_img_lab = cv2.cvtColor(modified_img, cv2.COLOR_BGR2LAB)

l_orig, a_orig, b_orig = cv2.split(original_img_lab)
l_mod, a_mod, b_mod = cv2.split(modified_img_lab)

a_mod = cv2.addWeighted(a_mod, 0.9, a_orig, 0.1, 0)
b_mod = cv2.addWeighted(b_mod, 0.9, b_orig, 0.1, 0)

modified_img_lab = cv2.merge((l_mod, a_mod, b_mod))
modified_img = cv2.cvtColor(modified_img_lab, cv2.COLOR_LAB2BGR)

mask = cv2.absdiff(original_img, modified_img)
gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
_, binary_mask = cv2.threshold(gray_mask, 30, 255, cv2.THRESH_BINARY)

binary_mask_cropped = np.zeros_like(binary_mask)
binary_mask_cropped[y:y+h, x:x+w] = binary_mask[y:y+h, x:x+w]

# Gaussian blur (feathering)
feathered_mask = cv2.GaussianBlur(binary_mask_cropped, (9, 9), 0)

blended_img = cv2.seamlessClone(modified_img, original_img, feathered_mask, center, cv2.NORMAL_CLONE)

cv2.imwrite('blended_output_v2.jpg', blended_img)
cv2.imshow('Blended Image', blended_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


<div class="alert alert-block alert-danger">Makes the photo more natural so it looks effective to some extent. However, I have some difficulty zoning in the modified region and some unwanted elements gets modified (like the beard unfortunately). Overall colours are also changed slightly but it is linked with the same problem : I am modifying more than I should</div>